In [40]:
import altair as alt
from vega_datasets import data
import pandas as pd
import numpy as np

In [386]:
# show how electricity prices went up over time
electricity_prices_eu = electricity_prices[electricity_prices['geo'] == 'EU27_2020']

# make line chart with modifications
# https://altair-viz.github.io/user_guide/customization.html
chart = alt.Chart(electricity_prices_eu,
        title = alt.Title(
            'Price of Electricity over time in the European Union',
            subtitle = ['Prices given in Purchasing Power Standard', 'which is a standardised form of currency'],
            subtitleFontSize=10)
    ).mark_line().encode(
    x = alt.X('TIME_PERIOD_dt:T', title = 'Time'),
    y = alt.Y('OBS_VALUE:Q', title = 'Purchasing Power Standar'),
    color = alt.Color("geo:N", title = None,
                      scale = alt.Scale(domain = ['EU27_2020'],
                                        range=['#134226'])).legend(None)
)

# create vertical line
line = alt.Chart(pd.DataFrame({'TIME_PERIOD_dt': ['2022-01-01']})).mark_rule(color='#f3892c').encode(
    x = 'TIME_PERIOD_dt:T'
)

# Add text for the vertical line
text1 = alt.Chart(pd.DataFrame({
    'TIME_PERIOD_dt': ['2022-01-01'],
    'label': ['Russian sanctions']
})).mark_text(
    align='left', 
    dx=5, 
    dy=-55,  # Adjust vertical position
).encode(
    x='TIME_PERIOD_dt:T',
    text='label:N'
)
text2 = alt.Chart(pd.DataFrame({
    'TIME_PERIOD_dt': ['2022-01-01'],
    'label': ['implemented']
})).mark_text(
    align='left', 
    dx=5, 
    dy=-45,  # Adjust vertical position
).encode(
    x='TIME_PERIOD_dt:T',
    text='label:N'
)

# add dot to graph
# https://altair-viz.github.io/user_guide/marks/point.html
dot_data = pd.DataFrame()
dot_data['time'] = ['2022-01-01']
dot_data['value'] = 0.255

dot = alt.Chart(dot_data).mark_point(filled=True, color = '#f3892c', size = 100).encode(
    x="time:T",
    y="value:Q"
)

chart + line + text1 + text2 + dot

alt.LayerChart(...)

In [240]:
# possible alternative for the previous graph

electricity_prices = pd.read_csv("nrg_pc_204_page_linear.csv")
electricity_prices = electricity_prices.drop(columns = ["DATAFLOW", "LAST UPDATE", "freq", "product", "nrg_cons", "unit", "tax", "currency", "OBS_FLAG"])

def convert_time_period(row):
    year, half = row.split('-')
    if half == 'S1':
        return pd.Timestamp(f'{year}-01-01')  # First half of the year
    else:
        return pd.Timestamp(f'{year}-07-01')  # Second half of the year

# Apply the conversion to the TIME_PERIOD column
electricity_prices['TIME_PERIOD_dt'] = electricity_prices['TIME_PERIOD'].apply(convert_time_period)

opacity_condition = alt.condition(
    alt.datum.geo == 'EU27_2020',  # If geo equals "EU27_2020"
    alt.value(1),  # Full opacity for "EU27_2020"
    alt.value(0.3)  # 0.5 opacity for all others
)

# https://github.com/vega/altair/discussions/2535
chart = alt.Chart(electricity_prices).mark_line().encode(
    x = 'TIME_PERIOD_dt:T',
    y = 'OBS_VALUE:Q',
    color = "geo:N",
    opacity=opacity_condition
)

chart.show()


alt.Chart(...)

In [385]:
# gas consumption in the EU 2016-2024
gas_consumption = pd.read_csv('gas_consumption.csv')
# fix the time in the dataset

# make line chart with modifications
# https://altair-viz.github.io/user_guide/customization.html
chart = alt.Chart(gas_consumption,
        title = alt.Title(
            'Gas Consumption in the EU Between 2016-2024')
    ).mark_line().encode(
    x = alt.X('TIME_PERIOD:T', title = 'Time'),
    y = alt.Y('OBS_VALUE:Q', title = 'Terajoule'),
    color = alt.Color("geo:N", title = None,
                      scale = alt.Scale(domain = ['EU27_2020'],
                                        range=['#134226'])).legend(None)
).properties(
    width = 400,
    height = 200
)

chart


alt.Chart(...)

In [387]:
# Share of russian imports of total consumption
import_russia = pd.read_csv("russia_gas_2021.csv")
consumption = pd.read_csv("consumption_2021.csv")

# create new table with ratio
import_ratio = pd.DataFrame()
import_ratio["country"] = import_russia["geo"]
import_ratio["ratio"] = (import_russia["OBS_VALUE"] / consumption["OBS_VALUE"]) * 100
import_ratio = import_ratio[import_ratio["country"] != "EU27_2020"]

# create country id column for visualization
# https://github.com/vega/altair/issues/2044
# https://en.wikipedia.org/wiki/ISO_3166-1_numeric
import_ratio["id"] = ["040", "056", "100", "196", "203", "276", "208", "233", "300", "724", "246", "250", "191", \
                      "348", "372", "380", "440", "442", "428", "470", "528", "616", "620", "642", "752", "705", \
                        "703"]

# create buckets for better interpretation
# https://stackoverflow.com/questions/45273731/binning-a-column-with-pandas

bins = [0, 20, 40, 60, 80, 100]
labels = ['0-20%', '21-40%', '41-60%', '61-80%', '81-100%']
import_ratio['bucket'] = pd.cut(import_ratio['ratio'], bins=bins, labels=labels, include_lowest=True)

# create map visualization
# https://altair-viz.github.io/altair-tutorial/notebooks/09-Geographic-plots.html
data_map = alt.topo_feature(data.world_110m.url, 'countries')

foreground = alt.Chart(data_map).mark_geoshape().encode(
    tooltip = [
        alt.Tooltip('properties.name:N', title='Country'),
        alt.Tooltip('ratio:Q', title = 'Ratio (%)', format = '.1f'),
        alt.Tooltip('bucket:N', title='Bucket')  
    ],
    # change n/a countries' color to grey
    # https://stackoverflow.com/questions/55229651/dealing-with-missing-values-nulls-in-altair-choropleth-map
    color = alt.Color('bucket:N', title = 'Percentage', 
                      scale = alt.Scale(domain = ['0-20%', '21-40%', '41-60%', '61-80%', '81-100%'],
                                        range = ['#feb99a', '#f3892c', '#be7b14', '#8c6c12', '#57560e']))
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(import_ratio, 'id', ['ids', 'ratio', 'bucket'])
).properties(
    width=500,
    height=300,
    title = 'Share of Russian Gas in Total Gas Consumption in 2021'
# get proper map of Europe
# https://stackoverflow.com/questions/61135952/vega-lite-altair-how-to-center-or-crop-a-map-of-europe
).project(
    type='mercator',
    scale= 350,                       
    center= [20,50],                     
    clipExtent= [[0, 0], [400, 300]],
# remove null 
).transform_filter(
    'isValid(datum.bucket)'
)

# make layered graph to show null values
# https://stackoverflow.com/questions/55229651/dealing-with-missing-values-nulls-in-altair-choropleth-map
background = alt.Chart(data_map).mark_geoshape(
        fill='lightgrey',
        stroke='white'
    ).project(
        type= 'mercator',
        scale= 350,                          # Magnify
        center= [20,50],                     # [lon, lat]
        clipExtent= [[0, 0], [400, 300]],    # [[left, top], [right, bottom]]
    ).properties(
        width=500, height=300
    )

background + foreground

alt.LayerChart(...)

In [231]:
# create electricity generation pie charts
electricity_generation = pd.read_csv('electricity_generation.csv')

# filter for EU countries
electricity_generation = electricity_generation[electricity_generation['EU'] == 1]

# drop unneccesary columns
electricity_generation = electricity_generation.drop(columns = ['Area type', 'Continent', 'Ember region', 'EU', 'OECD', 'G20', 'G7', 'ASEAN', 'Category', 'YoY absolute change', f'YoY % change'])

# create 2021 and 2024 datasets
electricity_generation_2021 = electricity_generation[electricity_generation['Date'].str.contains('2021')]
electricity_generation_2024 = electricity_generation[electricity_generation['Date'].str.contains('2024')]

# filter for relevant data
electricity_generation_2021 = electricity_generation_2021.loc[(electricity_generation_2021['Subcategory'] == 'Fuel') & (electricity_generation_2021['Unit'] == 'TWh')]
electricity_generation_2024 = electricity_generation_2024.loc[(electricity_generation_2024['Subcategory'] == 'Fuel') & (electricity_generation_2024['Unit'] == 'TWh')]

# sum up the energy generated for the entire EU
# https://stackoverflow.com/questions/40553002/group-by-two-columns-to-get-sum-of-another-column
electricity_europe_2024 = electricity_generation_2024.groupby(['Variable'], as_index=False)['Value'].sum()
electricity_europe_2021 = electricity_generation_2021.groupby(['Variable'], as_index=False)['Value'].sum()

# create other energy row
other_2021 = electricity_europe_2021[electricity_europe_2021['Variable'].isin(['Other Renewables', 'Other Fossil'])]['Value'].sum()
other_2024 = electricity_europe_2024[electricity_europe_2024['Variable'].isin(['Other Renewables', 'Other Fossil'])]['Value'].sum()


In [232]:
electricity_europe_2021 = electricity_europe_2021.loc[(electricity_europe_2021['Variable'] != 'Other Renewables') & (electricity_europe_2021['Variable'] != 'Other Fossil')]
electricity_europe_2024 = electricity_europe_2024.loc[(electricity_europe_2024['Variable'] != 'Other Renewables') & (electricity_europe_2024['Variable'] != 'Other Fossil')]

# append the new "Other" row 
# https://www.geeksforgeeks.org/how-to-add-one-row-in-an-existing-pandas-dataframe/
electricity_europe_2021 = electricity_europe_2021._append({'Variable': 'Other', 'Value': other_2021}, ignore_index=True)
electricity_europe_2024 = electricity_europe_2024._append({'Variable': 'Other', 'Value': other_2024}, ignore_index=True)


In [233]:
electricity_europe_2024

,Variable,Value
0,Bioenergy,68.62
1,Coal,163.99
2,Gas,247.47
3,Hydro,243.84
4,Nuclear,405.76
5,Solar,213.95
6,Wind,296.90
7,Other,49.17


In [388]:
color_scale_2021 = alt.Scale(
    domain=["Bioenergy", "Coal", "Gas", "Hydro", "Nuclear", "Solar", "Wind", "Other"],
    range=['#134226', '#57560e', '#f3892c', '#125f73', '#be7b14', '#feb99a', '#8fc6fa', '#8c6c12']
)

# Base chart
base_2021 = alt.Chart(electricity_europe_2021).encode(
    theta=alt.Theta("Value:Q", type = "quantitative", stack=True),
    color=alt.Color("Variable:N",
                    legend=alt.Legend(
                        title = None,
                        orient = "right",
                    ),
                    scale=color_scale_2021  # Apply the updated color scale
    ),
    tooltip=alt.Tooltip("Value:Q", title="Value")
).properties(
    title=alt.Title(text="Energy Sources in the EU in 2021", offset=10),
    height=200
)

# Pie chart
pie_2021 = base_2021.mark_arc(outerRadius=120)

# Text labels showing percentage
# https://github.com/vega/altair/issues/3069 
text_2021 = base_2021.mark_text(
    radius=140, size=12
).transform_joinaggregate(
    total_value='sum(Value)'
).transform_calculate(
    percentage="datum.Value / datum.total_value"
).encode(
    text=alt.Text('percentage:Q', format='.2%'),
)

# Combine pie chart and text labels
pie_2021 + text_2021

alt.LayerChart(...)

In [389]:
# https://github.com/vega/altair/issues/3069
# Define a color scale for the "Variable" categories
color_scale_2024 = alt.Scale(
    domain=["Bioenergy", "Coal", "Gas", "Hydro", "Nuclear", "Solar", "Wind", "Other"],
    range=['#134226', '#57560e', '#f3892c', '#125f73', '#be7b14', '#feb99a', '#8fc6fa', '#8c6c12']
)

# Base chart
base_2024 = alt.Chart(electricity_europe_2024).encode(
    theta=alt.Theta("Value:Q", type = "quantitative", stack=True),
    color=alt.Color("Variable:N",
                    legend=alt.Legend(
                        title = None,
                        orient = "right",
                    ),
                    scale=color_scale_2024  # Apply the updated color scale
    ),
    tooltip=alt.Tooltip("Value:Q", title="Value")
).properties(
    title=alt.Title(text="Energy Sources in the EU in 2024", offset=10),
    height=200
)

# Pie chart
pie_2024 = base_2024.mark_arc(outerRadius=120)

# Text labels showing percentage
# https://github.com/vega/altair/issues/3069 
text_2024 = base_2024.mark_text(
    radius=140, size=12
).transform_joinaggregate(
    total_value='sum(Value)'
).transform_calculate(
    percentage="datum.Value / datum.total_value"
).encode(
    text=alt.Text('percentage:Q', format='.2%')
)

# Combine pie chart and text labels
pie_2024 + text_2024

alt.LayerChart(...)

In [236]:
# bubble chart for change in share of renewable energy

# get share of renewable by country for 2021
renewable_2021 = electricity_generation_2021[electricity_generation_2021['Variable'].isin(['Bioenergy', 'Solar', 'Wind', 'Other Renewables'])]
renewable_country_2021 = renewable_2021.groupby(['Country code'], as_index=False)['Value'].sum()
renewable_country_2021_total = electricity_generation_2021.groupby(['Country code'], as_index=False)['Value'].sum()
renewable_country_2021.loc[renewable_country_2021['Country code'] == 'Malta', 'Value'] = 0.0001
renewable_country_2021['Total'] = renewable_country_2021_total['Value']
renewable_country_2021['Percent'] = (renewable_country_2021['Value'] / renewable_country_2021['Total']) * 100

# same for 2024
renewable_2024 = electricity_generation_2024[electricity_generation_2024['Variable'].isin(['Bioenergy', 'Solar', 'Wind', 'Other Renewables'])]
renewable_country_2024 = renewable_2024.groupby(['Country code'], as_index=False)['Value'].sum()
renewable_country_2024_total = electricity_generation_2024.groupby(['Country code'], as_index=False)['Value'].sum()
renewable_country_2024['Total'] = renewable_country_2024_total['Value']
renewable_country_2024['Percent'] = (renewable_country_2024['Value'] / renewable_country_2024['Total']) * 100

# change by country
change = pd.DataFrame()
change['Country'] = renewable_country_2024['Country code']
change['Change'] = (renewable_country_2024['Percent'] - renewable_country_2021['Percent']) / renewable_country_2021['Percent']
# take out Malta because it goes to infinity
change = change[change['Country'] != 'MLT']

change['Order'] = [2,10,18,22,26,20,6,4,13,3,17,19,11,5,14,16,1,12,8,23,15,25,24,9,21,7]
change


,Country,Change,Order
0,AUT,0.622359,2
1,BEL,0.693865,10
2,BGR,1.730218,18
3,CYP,0.531090,22
4,CZE,0.524462,26
5,DEU,0.469573,20
6,DNK,0.187083,6
7,ESP,0.301540,4
8,EST,0.883717,13
9,FIN,0.175777,3


In [396]:
# change in the share of renewable energy over time
# https://altair-viz.github.io/gallery/scatter_with_labels.html
# https://www.geeksforgeeks.org/how-to-remove-the-altair-y-axis-a-comprehensive-guide/
colors = ['#134226', '#57560e', '#f3892c', '#125f73', '#be7b14', '#feb99a', '#8fc6fa', '#8c6c12']
chart = alt.Chart(change).mark_point().encode(
    x=alt.X('Order', axis=None),
    y=alt.Y('Change', title = 'Percentage change'),
    color = alt.Color('Country', title = None, scale=alt.Scale(range=colors)).legend(None)
).properties(
    width=250,
    height=500,
    title = ['Change in Share of Renewable Energies', 'in Energy Production']
)

text = chart.mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    text='Country'
)

line = alt.Chart(pd.DataFrame({'Change': [0]})).mark_rule(color='#f3892c').encode(
    y = 'Change'
)

chart + text + line

alt.LayerChart(...)

In [392]:
# amount of natural gas import by partner country 
import_2021 = pd.read_csv('import_country_2021.csv')
import_2024 = pd.read_csv('import_country_2024.csv')

import_2024 = import_2024.groupby('partner', as_index=False)['OBS_VALUE'].sum()

# group the two datasets together
import_2024['TIME_PERIOD'] = ['2024', '2024', '2024', '2024', '2024', '2024', '2024']
imports = import_2021
imports = imports._append(import_2024)

# calculate other imports from Total
imports = imports._append({'partner': 'Other', 'OBS_VALUE': 41458.74, 'TIME_PERIOD': 2021}, ignore_index=True)
imports = imports._append({'partner': 'Other', 'OBS_VALUE': 65187.67, 'TIME_PERIOD': '2024'}, ignore_index=True)
imports = imports.loc[imports['partner'] != 'TOTAL']

country_names = {'RU' : 'Russia', 'NO':'Norway', 'DZ':'Algeria', 'UK': 'United Kingdom', 'US':'United States', \
                 'QA':'Qatar', 'Other':'Other'}
imports = imports.replace({'partner':country_names})

# https://altair-viz.github.io/gallery/grouped_bar_chart.html
# https://stackoverflow.com/questions/78219650/custom-ordering-in-a-grouped-bar-chart-not-applied-in-altair
alt.Chart(imports).mark_bar().encode(
    x=alt.X(
        'TIME_PERIOD:O',
            axis=alt.Axis(labelAngle=-0, title=None,)
        ),
    y= alt.Y(
        'OBS_VALUE:Q',
        title = 'Million cubic meters'),
    color = alt.Color('TIME_PERIOD:N', title = None,
                      scale = alt.Scale(domain = [2021, '2024'],
                                        range = ['#be7b14', '#f3892c'])).legend(None),
    column=alt.Column(
        'partner:N',
        header=alt.Header(title=None, labelOrient='bottom'),
        sort = ['Russia', 'Norway', 'Algeria', 'United States', 'Qatar', 'United Kingdom', 'Other']
    ),
    xOffset='year:O',
).properties(
    title = ['Imports of Natural Gas to the European Union','by Partner Country']
)


alt.Chart(...)

In [376]:
# gas flow by various channels
route = pd.read_csv('gas_lines.csv', sep = ';')
route
# https://altair-viz.github.io/gallery/simple_stacked_area_chart.html
# format the data so it can work as a stacked chart
nordstream = pd.DataFrame()
nordstream['Time'] = route['Year']
nordstream['Volume'] = route['Nordstream']
nordstream['Pipeline'] = 'Nordstream'

ukraine = pd.DataFrame()
ukraine['Time'] = route['Year']
ukraine['Volume'] = route['Ukraine Gas Transit']
ukraine['Pipeline'] = 'Ukraine Gas Transit'

yamal = pd.DataFrame()
yamal['Time'] = route['Year']
yamal['Volume'] = route['Yamal']
yamal['Pipeline'] = 'Yamal'

turk = pd.DataFrame()
turk['Time'] = route['Year']
turk['Volume'] = route['Turkstream']
turk['Pipeline'] = 'Turkstream'

pipelines = nordstream._append(ukraine)
pipelines = pipelines._append(yamal)
pipelines = pipelines._append(turk)

pipelines['Time'] = pd.to_datetime(pipelines['Time'], dayfirst=True)

In [395]:
stacked = alt.Chart(pipelines).mark_area().encode(
    x="Time:T",
    y=alt.Y("Volume:Q", title = 'Million cubic meters per week'),
    color=alt.Color("Pipeline:N",
                    scale = alt.Scale(range = ['#8c6c12', '#be7b14', '#f3892c', '#feb99a'],
                                      domain = ['Nordstream', 'Turkstream', 'Ukraine Gas Transit',
                                                'Yamal']))
).properties(
    title = 'Russian gas exports to the EU 2021-2024',
    height = 200,
    width = 500
)
stacked

alt.Chart(...)

In [374]:
pipelines['Volume'] = pipelines['Volume'].fillna(method='ffill')
pipelines['Volume'] = pipelines['Volume'].interpolate(method='linear')


/var/folders/n2/bxxydqhj2_v83dv6v9qr14j40000gn/T/ipykernel_23656/1258328994.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pipelines['Volume'] = pipelines['Volume'].fillna(method='ffill')
/var/folders/n2/bxxydqhj2_v83dv6v9qr14j40000gn/T/ipykernel_23656/1258328994.py:2: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  pipelines['Volume'] = pipelines['Volume'].interpolate(method='linear')
